In [1]:
import pandas as pd
import numpy as np

import pickle
import importlib

import nltk
#nltk.data.path.append('C:\\Users\\della\\anaconda3\\nltk-data')

from sklearn.model_selection import train_test_split

import pre_processing
#importlib.reload(pre_processing)

import text_vectorization
#importlib.reload(text_vectorization)

<module 'text_vectorization' from 'c:\\Users\\della\\Desktop\\arXiv_classification\\text_vectorization.py'>

In [2]:
arxiv_data = pd.read_csv('data/arxiv-dataset-cat1.csv')

In [3]:
train_set, test_set = train_test_split(arxiv_data, test_size = 0.3, stratify = arxiv_data['label'], random_state = 19)

In [4]:
#train_set_processed = pre_processing.dataPreProcessing(train_set)
#train_set_processed.to_csv('data/train-set-cat1-processed.csv', index = False)
train_set = pd.read_csv('data/train-set-cat1-processed.csv')

In [5]:
vocabulary_size = 50000
words_per_sentence = 200

text_vectorizer = text_vectorization.createTextVectorizer(vocabulary_size, words_per_sentence, train_set['text'])
train_set = text_vectorization.textVectorization(train_set['text'], text_vectorizer)

In [167]:
with open('../glove-embedding.pkl', 'rb') as file:
    words_embedding = pickle.load(file)

In [170]:
embedding_dim = 100
embedding_matrix = np.zeros((vocabulary_size + 2, embedding_dim))

for i, word in enumerate(text_vectorizer.get_vocabulary()):

    word_embedding_repr = words_embedding.get(word)

    if word_embedding_repr is not None:
        embedding_matrix[i] = word_embedding_repr
    else:
        embedding_matrix[i] = np.zeros(embedding_dim)

In [186]:
print('Length of embedding vocabulary:', len([item for item in embedding_matrix[:,0] if item != 0]))

Length of embedding vocabulary: 32395


In [187]:
from keras.layers import Embedding
from keras.initializers import Constant

embedding_layer = Embedding(
    vocabulary_size + 2, embedding_dim,
    embeddings_initializer = Constant(embedding_matrix),
    embeddings_regularizer = None
)